# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [ ]:
# Only needed for Udacity workspace

import importlib.util
import sys

from onnxruntime.transformers.models.stable_diffusion.diffusion_models import BaseModel

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [30]:
import chromadb
from dotenv import load_dotenv
from pydantic import BaseModel, Field, ConfigDict

from lib.agents import Agent
from lib.llm import LLM
from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage
from lib.tooling import tool

In [31]:
load_dotenv()

True

In [32]:
chroma_client = chromadb.PersistentClient(path="chromadb")
collection = chroma_client.get_collection("udaplay")

In [33]:
class Game(BaseModel):
    """Game information"""
    model_config = ConfigDict(
        frozen=True,
    )

    Platform: str = Field(
        ...,
        description="Platform name of the game",
        alias="Platform",
        examples=["Game Boy", "Playstation 5", "Xbox 360"]
    )
    Name: str = Field(
        ...,
        description="Name of the Game",
        alias="Name",
        examples=["Super Mario World", "Pokémon Red and Blue", "Gran Turismo"]
    )
    YearOfRelease: int = Field(
        ...,
        description="Year when that game was released for that platform",
        alias="YearOfRelease",
        examples=[1990, 2001, 2010]
    )
    Description: str = Field(
        ...,
        description="Additional details about the game",
        alias="Description",
        examples=["A classic platformer where Mario embarks on a quest to save Princess Toadstool and Dinosaur Land from Bowser."]
    )

In [34]:
@tool
def retrieve_game(query: str, num_of_results = 5) -> list[Game]:
    """
    Semantic search: Finds most results in the vector DB

    args:
    - query: a question about game industry.
    - num_of_results: number of results to return

    You'll receive results as list. Each element contains:
    - Platform: like Game Boy, Playstation 5, Xbox 360...
    - Name: Name of the Game
    - YearOfRelease: Year when that game was released for that platform
    - Description: Additional details about the game
    """
    response = collection.query(query_texts=[query], n_results=num_of_results)

    if not (metadata := response.get("metadatas", [[]])[0]):
        return metadata

    return [Game(**meta) for meta in metadata]

#### Evaluate Retrieval Tool

In [ ]:
# TODO: Create evaluate_retrieval tool
# You might use an LLM as judge in this tool to evaluate the performance
# You need to prompt that LLM with something like:
# "Your task is to evaluate if the documents are enough to respond the query. "
# "Give a detailed explanation, so it's possible to take an action to accept it or not."
# Use EvaluationReport to parse the result
# Tool Docstring:
#    Based on the user's question and on the list of retrieved documents, 
#    it will analyze the usability of the documents to respond to that question. 
#    args: 
#    - question: original question from user
#    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
#    The result includes:
#    - useful: whether the documents are useful to answer the question
#    - description: description about the evaluation result
@tool
def evaluate_retrieval(question: str, retrieved_docs: list[Game]) -> EvaluationReport:

#### Game Web Search Tool

In [ ]:
# TODO: Create game_web_search tool
# Please use Tavily client to search the web
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - question: a question about game industry. 

### Agent

In [ ]:
# TODO: Create your Agent abstraction using StateMachine
# Equip with an appropriate model
# Craft a good set of instructions 
# Plug all Tools you developed

In [ ]:
# TODO: Invoke your agent
# - When Pokémon Gold and Silver was released?
# - Which one was the first 3D platformer Mario game?
# - Was Mortal Kombat X realeased for Playstation 5?

### (Optional) Advanced

In [ ]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes